In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.svm as svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics 
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score, cross_validate
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

#추가
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, fbeta_score

In [104]:
train_df = pd.read_csv('../EDA/scaled_data.csv')
train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
train_df.head(1)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,직업,가족 구성원 수,산업군,나이,가입연수,도시구분,월간 수입
0,0,2,1,1,3,0,0,3,1,0,1,4,5,1,2,3,0


In [105]:
train_copy = train_df.copy()

In [ ]:
# 어느 산업군에서 어느 직업이 연체 여부가 높은지 확인

# 직업 수, 산업군 수의 편중을 완화하기 위하여 각 직업 별 연체 여부로 연체율 확인/ 각 산업군 별 연체 여부로 연체율 확인
# 위에서 확인한 각 산업군의 연체율을 총 평균연체율(10.71%)보다 높은 산업군 15개(총 33개의 직업 중)
# 각 산업군의 연체율을 총 평균연체율(10.71%)보다 높은 직업 8개(총 19개의 직업 중)

## 종사하는 산업군의 전체 인원수를 확인하여 너무 적은 경우 제외
## 직업에 종사하는 인원수를 확인하여 너무 적은 경우 제외

# 위의 사항 고려하여 top3 산업군과 top4 직업 선정
# 이를 결합하여 1위 산업군- 1위 직업, 2위 직업, 3위 직업 ... 3위 산업군- 1위 직업, 2위 직업, 3위 직업 비교
# 어느 파생변수가 가장 유의한지 -> 그 산업군에서 그 직업이 연체율이 높다고 예측 확인

In [ ]:
# 산업군 Top3 : 8(레스토랑-0.195%), 0(건설업-0.155%), 24(자영업-0.137%)
# 직업 Top4 : 15(저임금 노동자-0.237%), 12(운전자-0.150%), 17(보안업계종사자-0.149%), 5(단순노동자-0.140%)


In [106]:
# 산업군_직업 컬럼 생성 함수
def create_job_column(df, ind_value, job_value):

#    새로운 컬럼을 생성하고 특정 조건에 따라 값을 할당하는 함수

#    Parameters:
#    - df: 데이터프레임
#    - job_value: 직업 구분코드
#    - ind_value : 산업군 구분코드

#    Returns:
#    - df: 새로운 컬럼을 추가한 데이터프레임
    
  new_column_name = f"{ind_value}_{job_value}"
  # 조건에 맞는 행에 1을 할당하여 새로운 컬럼에 추가
  df[new_column_name] = (df['산업군'] == ind_value) & (df['직업'] == job_value)
  # True/False를 1/0으로 변환
  df[new_column_name] = df[new_column_name].astype(int)
    
  return df
# 함수 사용 예시
# create_job_column(your_df, ind_value, job_value)

In [107]:
# 산업군 8_직업 컬럼 생성
create_job_column(train_copy, 8, 15)
create_job_column(train_copy, 8, 12)
create_job_column(train_copy, 8, 17)
create_job_column(train_copy, 8, 5)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,...,가족 구성원 수,산업군,나이,가입연수,도시구분,월간 수입,8_15,8_12,8_17,8_5
0,0,2,1,1,3,0,0,3,1,0,...,4,5,1,2,3,0,0,0,0,0
1,0,1,1,0,1,1,0,3,1,0,...,2,16,2,1,1,3,0,0,0,0
2,0,2,0,1,1,0,0,3,1,0,...,2,16,1,1,1,1,0,0,0,0
3,0,2,1,0,2,0,0,3,1,0,...,3,17,1,1,3,2,0,0,0,0
4,0,2,0,1,1,0,0,3,1,0,...,2,16,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59984,0,2,0,1,3,0,1,3,1,0,...,1,5,3,2,1,0,0,0,0,0
59985,1,1,1,1,1,0,0,3,1,0,...,2,24,2,0,1,3,0,0,0,0
59986,1,2,0,0,1,1,1,3,1,0,...,1,22,0,1,1,2,0,0,0,0
59987,0,2,1,1,1,0,3,3,1,0,...,1,19,2,1,1,1,0,0,0,0


In [108]:
# 산업군 0_직업 컬럼 생성
create_job_column(train_copy, 0, 15)
create_job_column(train_copy, 0, 12)
create_job_column(train_copy, 0, 17)
create_job_column(train_copy, 0, 5)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,...,도시구분,월간 수입,8_15,8_12,8_17,8_5,0_15,0_12,0_17,0_5
0,0,2,1,1,3,0,0,3,1,0,...,3,0,0,0,0,0,0,0,0,0
1,0,1,1,0,1,1,0,3,1,0,...,1,3,0,0,0,0,0,0,0,0
2,0,2,0,1,1,0,0,3,1,0,...,1,1,0,0,0,0,0,0,0,0
3,0,2,1,0,2,0,0,3,1,0,...,3,2,0,0,0,0,0,0,0,0
4,0,2,0,1,1,0,0,3,1,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59984,0,2,0,1,3,0,1,3,1,0,...,1,0,0,0,0,0,0,0,0,0
59985,1,1,1,1,1,0,0,3,1,0,...,1,3,0,0,0,0,0,0,0,0
59986,1,2,0,0,1,1,1,3,1,0,...,1,2,0,0,0,0,0,0,0,0
59987,0,2,1,1,1,0,3,3,1,0,...,1,1,0,0,0,0,0,0,0,0


In [109]:
# 산업군 24_직업 컬럼 생성
create_job_column(train_copy, 24, 15)
create_job_column(train_copy, 24, 12)
create_job_column(train_copy, 24, 17)
create_job_column(train_copy, 24, 5)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,...,8_17,8_5,0_15,0_12,0_17,0_5,24_15,24_12,24_17,24_5
0,0,2,1,1,3,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,1,1,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,1,1,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,1,0,2,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,1,1,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59984,0,2,0,1,3,0,1,3,1,0,...,0,0,0,0,0,0,0,0,0,0
59985,1,1,1,1,1,0,0,3,1,0,...,0,0,0,0,0,0,0,1,0,0
59986,1,2,0,0,1,1,1,3,1,0,...,0,0,0,0,0,0,0,0,0,0
59987,0,2,1,1,1,0,3,3,1,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59989 entries, 0 to 59988
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   TARGET      59989 non-null  int64
 1   성별          59989 non-null  int64
 2   차량 소유 여부    59989 non-null  int64
 3   부동산 소유 여부   59989 non-null  int64
 4   수입 유형       59989 non-null  int64
 5   최종 학력       59989 non-null  int64
 6   결혼 여부       59989 non-null  int64
 7   주거 형태       59989 non-null  int64
 8   휴대전화 소유 여부  59989 non-null  int64
 9   이메일 소유 여부   59989 non-null  int64
 10  직업          59989 non-null  int64
 11  가족 구성원 수    59989 non-null  int64
 12  산업군         59989 non-null  int64
 13  나이          59989 non-null  int64
 14  가입연수        59989 non-null  int64
 15  도시구분        59989 non-null  int64
 16  월간 수입       59989 non-null  int64
 17  8_15        59989 non-null  int32
 18  8_12        59989 non-null  int32
 19  8_17        59989 non-null  int32
 20  8_5         59989 non-null  

In [111]:
train_copy['0_15'].unique()

array([0, 1])

In [112]:
# 파생변수 '가족크기범주' 생성
train_copy['가족크기범주'] = pd.cut(train_copy['가족 구성원 수'], bins=[0, 2, 4,float('inf')],  
                    labels=['0','1', '2'])

In [113]:
# 데이터타입 category -> int 변경 
train_copy['가족크기범주'] = train_copy['가족크기범주'].dropna().astype(int)

In [114]:
# 파생변수 'combinedFY' 생성
# 조건 설정
conditions = [
(train_copy['가족크기범주'] == 0) & (train_copy['가입연수'] == 0),
(train_copy['가족크기범주'] == 0) & (train_copy['가입연수'] == 1),
(train_copy['가족크기범주'] == 0) & (train_copy['가입연수'] == 2),
(train_copy['가족크기범주'] == 0) & (train_copy['가입연수'] == 3),

(train_copy['가족크기범주'] == 1) & (train_copy['가입연수'] == 0),
(train_copy['가족크기범주'] == 1) & (train_copy['가입연수'] == 1),
(train_copy['가족크기범주'] == 1) & (train_copy['가입연수'] == 2),
(train_copy['가족크기범주'] == 1) & (train_copy['가입연수'] == 3),


(train_copy['가족크기범주'] == 2) & (train_copy['가입연수'] == 0),
(train_copy['가족크기범주'] == 2) & (train_copy['가입연수'] == 1),
(train_copy['가족크기범주'] == 2) & (train_copy['가입연수'] == 2),
(train_copy['가족크기범주'] == 2) & (train_copy['가입연수'] == 3),
]
# 할당할 값 설정
values = [0, 1, 2, 3, 4, 5, 6, 7, 8,9,10,11]

# np.select를 사용하여 조건에 맞는 값을 'y' 컬럼에 할당
train_copy['combinedfY'] = np.select(conditions, values, default=0)
#train_copy = train_copy.drop(['차량 소유 여부', '부동산 소유 여부', '주거 형태', '근속연수'], axis=1)
train_copy.head(2)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,...,0_15,0_12,0_17,0_5,24_15,24_12,24_17,24_5,가족크기범주,combinedfY
0,0,2,1,1,3,0,0,3,1,0,...,0,0,0,0,0,0,0,0,1,6
1,0,1,1,0,1,1,0,3,1,0,...,0,0,0,0,0,0,0,0,0,1


In [115]:
# 파생변수 'age_income' 생성
# 임금수준 & 나이활용한 파생변수
conditions1 = [
(train_copy['나이'] == 0) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 0) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 0) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 0) & (train_copy['월간 수입'] == 3),

(train_copy['나이'] == 1) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 1) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 1) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 1) & (train_copy['월간 수입'] == 3),

(train_copy['나이'] == 2) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 2) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 2) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 2) & (train_copy['월간 수입'] == 3),

(train_copy['나이'] == 3) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 3) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 3) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 3) & (train_copy['월간 수입'] == 3),

(train_copy['나이'] == 4) & (train_copy['월간 수입'] == 0),
(train_copy['나이'] == 4) & (train_copy['월간 수입'] == 1),
(train_copy['나이'] == 4) & (train_copy['월간 수입'] == 2),
(train_copy['나이'] == 4) & (train_copy['월간 수입'] == 3),

]
# 할당할 값 설정
values = [0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
# np.select를 사용하여 조건에 맞는 값을 'y' 컬럼에 할당
train_copy['age_income'] = np.select(conditions1, values, default=0)
train_copy.head(2)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,...,0_12,0_17,0_5,24_15,24_12,24_17,24_5,가족크기범주,combinedfY,age_income
0,0,2,1,1,3,0,0,3,1,0,...,0,0,0,0,0,0,0,1,6,4
1,0,1,1,0,1,1,0,3,1,0,...,0,0,0,0,0,0,0,0,1,11


In [116]:
# 파생변수 'home_shape' 생성
# 부동산 소유 여부와 주거형태 활용한 변수
conditions = [
(train_copy['부동산 소유 여부'] == 0) & (train_copy['주거 형태'] == 0),
(train_copy['부동산 소유 여부'] == 0) & (train_copy['주거 형태'] == 1),
(train_copy['부동산 소유 여부'] == 0) & (train_copy['주거 형태'] == 2),
(train_copy['부동산 소유 여부'] == 0) & (train_copy['주거 형태'] == 3),
(train_copy['부동산 소유 여부'] == 1) & (train_copy['주거 형태'] == 0),
(train_copy['부동산 소유 여부'] == 1) & (train_copy['주거 형태'] == 1),
(train_copy['부동산 소유 여부'] == 1) & (train_copy['주거 형태'] == 2),
(train_copy['부동산 소유 여부'] == 1) & (train_copy['주거 형태'] == 3),
]
# 할당할 값 설정
values = [0,1, 2, 3, 4, 5, 6, 7]
# np.select를 사용하여 조건에 맞는 값을 'y' 컬럼에 할당
train_copy['home_shape'] = np.select(conditions, values, default=0)

In [117]:
# 파생변수 'car_home' 생성
# 차량 소유 여부와 부동산 소유 여부 활용한 변수
conditions = [
(train_copy['차량 소유 여부'] == 0) & (train_copy['부동산 소유 여부'] == 0),
(train_copy['차량 소유 여부'] == 0) & (train_copy['부동산 소유 여부'] == 1),
(train_copy['차량 소유 여부'] == 1) & (train_copy['부동산 소유 여부'] == 0),
(train_copy['차량 소유 여부'] == 1) & (train_copy['부동산 소유 여부'] == 1)
]
# 할당할 값 설정
values = [0,1,2,3]
# np.select를 사용하여 조건에 맞는 값을 'y' 컬럼에 할당
train_copy['car_home'] = np.select(conditions, values, default=0)

In [ ]:
# 카이제곱 검정은 파생변수가 범주형일 때 유의미한지 검증하는 대표적인 지표.
# 결과로 나온 출력에서 p-value가 0.05 미만이면서 chi2가 가장 높은 파생변수가
# 카이제곱 검정으로 봤을때 가장 의미가 있는 피쳐.

# 일단 가설을 설정했다면 그 가설에 필요한 변수만 빼서 카이제곱 검정을 돌려. 
# 그래서 0.05이상 나오면 그 가설과 파생변수는 유의미하니 사용 가능.
# 그리고 이것도 통계적 요소 대립가설을 언급하며 논리를 펼칠 것.

In [119]:
train_copy.columns

Index(['TARGET', '성별', '차량 소유 여부', '부동산 소유 여부', '수입 유형', '최종 학력', '결혼 여부',
       '주거 형태', '휴대전화 소유 여부', '이메일 소유 여부', '직업', '가족 구성원 수', '산업군', '나이',
       '가입연수', '도시구분', '월간 수입', '8_15', '8_12', '8_17', '8_5', '0_15', '0_12',
       '0_17', '0_5', '24_15', '24_12', '24_17', '24_5', '가족크기범주',
       'combinedfY', 'age_income', 'home_shape', 'car_home', '소득 4분위',
       '소득 5분위', '소득 10분위'],
      dtype='object')

In [120]:
# p-value값 확인
from scipy.stats import chi2_contingency
chi2_results = {}
categorical_features = ['8_15', '8_12', '8_17', '8_5', '0_15', '0_12',
       '0_17', '0_5', '24_15', '24_12', '24_17', '24_5', '가족크기범주',
       'combinedfY', 'age_income', '소득 4분위',
       '소득 5분위', '소득 10분위']

# 나의 가설은 이렇다.
# 비교적 나이가 젊은 층 중 기혼자가 미혼자에 비해 재정 관리 능력이 더 뛰어날 것.
# 특히 월간 수입이 이러한 범위에 있는 사람은 연체 확률이 더 높거나 낮을 것.
# 그럼 내 가설에 필요한 피처들을 카이제곱 검정에 넣어.

In [121]:
# p-value값 확인
for feature in categorical_features:
    contingency_table = pd.crosstab(train_copy[feature], train_copy['TARGET'])
    
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    chi2_results[feature] = {'chi2': chi2, 'p-value': p}


chi2_results

{'8_15': {'chi2': 0.11114709784753601, 'p-value': 0.7388419416561753},
 '8_12': {'chi2': 6.278177989877958e-09, 'p-value': 0.9999367796533193},
 '8_17': {'chi2': 1.3521077297086408, 'p-value': 0.2449099784118336},
 '8_5': {'chi2': 2.1830295476658343, 'p-value': 0.13953954646675246},
 '0_15': {'chi2': 7.470017772758131, 'p-value': 0.0062734932102280145},
 '0_12': {'chi2': 0.4325710874980171, 'p-value': 0.5107300441169389},
 '0_17': {'chi2': 0.40357025573502747, 'p-value': 0.5252511644331217},
 '0_5': {'chi2': 56.32566547441838, 'p-value': 6.140859476987638e-14},
 '24_15': {'chi2': 49.179635398679615, 'p-value': 2.335629271922601e-12},
 '24_12': {'chi2': 55.20834793225451, 'p-value': 1.0840734935717556e-13},
 '24_17': {'chi2': 0.7682655616325583, 'p-value': 0.380754150075575},
 '24_5': {'chi2': 34.97785861903275, 'p-value': 3.3347585629109594e-09},
 '가족크기범주': {'chi2': 38.69955171882303, 'p-value': 3.949108975068367e-09},
 'combinedfY': {'chi2': 176.77555679594238, 'p-value': 1.0937242163

In [ ]:
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test,pred_proba)
    
# 아래 추가함 (pr_score, f2 score, gmean)
# pr_score : 모델이 양성 클래스를 얼마나 정확하게 찾아내는지에 대한 정보를 제공(클래스 불균형이 존재할 때 유용)
# fbeta_score : 정밀도(Precision)와 재현율(Recall)의 조화 평균을 계산하는 지표.재현율에 더 큰 가중치를 두는데, 이는 False Negatives를 최소화하는 데 중점
# G-Mean : Sensitivity와 Specificity의 조화 평균. 클래스 간의 불균형을 고려하여 모델의 성능을 측정하며, 특히 Positive 클래스의 예측 성능이 중요한 경우에 유용
    confusion = confusion_matrix(y_test, y_pred)
    pr_score = average_precision_score(y_test, y_pred)
    f2 = fbeta_score(y_test, y_pred, beta=2)

    # G-mean 계산
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    tpr = tp / (tp + fn)  # True Positive Rate
    tnr = tn / (tn + fp)  # True Negative Rate
    gmean = np.sqrt(tpr * tnr)

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
    print('Fbeta :{:.4f}'.format(f2))
    print('평균 정밀도 : {:.4f}'.format(pr_score))
    print('gmean : {:.4f}'.format(gmean))

In [ ]:
# feature와 label 분리
feature = df.drop('TARGET', axis=1)
label = df['TARGET']

In [ ]:
# train test 분리
X_train, X_test, y_train , y_test = train_test_split(feature, label, test_size=0.3 , random_state=42)

In [ ]:
# 언더샘플링 - 소수라벨과 다수라벨 비율 1:1

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
X_train, y_train = undersample.fit_resample(X_train, y_train)

In [ ]:
# 오버샘플링 - 소수라벨과 다수라벨 비율 1:1 / 오버 샘플링이 좋음 투터님 피셜

from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
# SMOTE

from imblearn.over_sampling import SMOTE
smote_sample = SMOTE(sampling_strategy='minority') 
X_train, y_train = smote_sample.fit_resample(X_train, y_train)

In [ ]:
# 1. Decision Tree를 이용한 학습

dt = DecisionTreeClassifier(random_state=43)

# 교차검증을 통한 과적합 or 과소적합 여부 확인
cross_val = cross_val_score(dt , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))

In [ ]:
# GridSearchCV를 적용해 Decision Tree의 교차검증 및 하이퍼파라미터 튜닝

parameters = {'max_depth' : [2, 3, 4, 5],
             'min_samples_split' : [1, 3, 5, 7, 9]}

grid_dt = GridSearchCV(dt, param_grid = parameters, cv=3, refit=True)
grid_dt.fit(X_train, y_train)

dt = grid_dt.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_dt.best_params_}")
print(f"최고 예측 정확도: {grid_dt.best_score_:.4f}")

In [ ]:
# 임곗값 조정을 통한 recall값 향상
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
pred_proba = dt.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test, custom_pred, pred_proba)

In [ ]:
# 2. RandomForest를 이용한 학습

rf = RandomForestClassifier(random_state=43)

cross_val = cross_val_score(rf , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))

In [ ]:
# 하이퍼파라미터 조정

parameters = {
    'max_depth' : [6, 8, 12],
    'min_samples_split' : [16, 24]
}

grid_rf = GridSearchCV(rf, param_grid = parameters, cv=3, refit=True)
grid_rf.fit(X_train, y_train)

rf = grid_rf.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_rf.best_params_}")
print(f"최고 예측 정확도: {grid_rf.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 적용

rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred_proba = rf.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test, custom_pred, pred_proba)

In [ ]:
# 3. Logistic regrssion 을 이용한 학습

lr = LogisticRegression(random_state=43)

cross_val = cross_val_score(lr, X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))


In [ ]:
parameters = {'penalty': ['l2','l1'],
          'C':[0.01,0.1,1,10]}

grid_lr = GridSearchCV(lr, param_grid = parameters, cv=3, refit=True)
grid_lr.fit(X_train, y_train)

lr = grid_lr.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_lr.best_params_}")
print(f"최고 예측 정확도: {grid_lr.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

lr.fit(X_train, y_train)
pred = lr.predict(X_test)
pred_proba = lr.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

In [ ]:
# 4. KNN 을 이용한 학습

knn = KNeighborsClassifier()
# knn에는 무작위성 추출(random_state)를 하지 않음.

cross_val = cross_val_score(knn , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))

In [ ]:
# 하이퍼파라미터 조정

parameters = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance']
          }

grid_knn = GridSearchCV(knn, param_grid = parameters, cv=3, refit=True)
grid_knn.fit(X_train, y_train)

knn = grid_knn.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_knn.best_params_}")
print(f"최고 예측 정확도: {grid_knn.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

knn.fit(X_train, y_train)
pred = knn.predict(X_test)
pred_proba = knn.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

In [ ]:
# 5. Lightgbm을 이용한 학습

lgbm= LGBMClassifier(random_state=43)

cross_val = cross_val_score(lgbm, X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))

In [ ]:
# 하이퍼파라미터 조정

parameters = {'n_neighbors': [100, 500],
              'learning_rate': [0.05, 0.1]
          }

grid_lgbm = GridSearchCV(lgbm, param_grid = parameters, cv=2, verbose=1, refit=True)
grid_lgbm.fit(X_train, y_train)

lgbm = grid_lgbm.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_lgbm.best_params_}")
print(f"최고 예측 정확도: {grid_lgbm.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
pred_proba = lgbm.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

In [ ]:
# 6. XGboost 
xgb = XGBClassifier(random_state=43)

cross_val = cross_val_score(xgb, X_train, y=y_train, cv=5, scoring='accuracy')
print(np.mean(cross_val))

In [ ]:
# 하이퍼파라미터 조정
parameters = {'learning_rat': [0.01, 0.1],  
              'n_estimators': [50, 100, 200]
          }

grid_xgb = GridSearchCV(xgb, param_grid=parameters, cv=2, verbose=1, refit=True)
grid_xgb.fit(X_train, y_train)

xgb = grid_xgb.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_xgb.best_params_}")
print(f"최고 예측 정확도: {grid_xgb.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
pred_proba = xgb.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)